<a href="https://colab.research.google.com/github/selector-ai/s2ai-infra/blob/main/fine-tuning/google-collab/Llama-3-8b-Instruct-finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install pre-requisite packages and add configuration for fine-tuning


In [1]:
# Set fine tuning input parameters
debug = True
model_name = "hf_model_name"
load_in_4bit = False
max_seq_length = 2048
hf_token = "hf_token"
lora_rank = 16
lora_alpha = 16
dtype = None
dataset_name = "hf_dataset_link"
save_local = False
upload_to_hf = ""
merge16_and_upload = ""
merge4_and_upload = ""

In [3]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [4]:
# Instantiate Language Model
from unsloth import FastLanguageModel
import torch
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = hf_token, # gated model
)

# Add Lora Adapter
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = lora_alpha,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/798 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/143 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.3k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Prepare dataset

In [5]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

unsloth_template = \
      "{% set loop_messages = messages %}"\
      "{% for message in loop_messages %}"\
          "{% if message['role'] == 'function_response' %}"\
              "{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n' + message['content'] | trim + '\\n\\nNow, make use of the above information - if relevant - to assist with the user\\'s request.' + '<|eot_id|>' %}"\
          "{% elif message['role'] == 'user' %}"\
              "{% set content = '<|start_header_id|>user<|end_header_id|>\\n\\n' + message['content'] | trim + '<|eot_id|>' %}"\
          "{% elif message['role'] == 'assistant' %}"\
              "{% set content = '<|start_header_id|>assistant<|end_header_id|>\\n\\n' + message['content'] | trim + '<|eot_id|>' %}"\
          "{% else %}"\
              "{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\\n\\n' + message['content'] | trim + '<|eot_id|>' %}"\
          "{% endif %}"\
          "{% if loop.index0 == 0 %}"\
              "{% set content = bos_token + content %}"\
          "{% endif %}"\
          "{{ content }}"\
      "{% endfor %}"\
      "{% if add_generation_prompt %}"\
          "{{ '<|start_header_id|>assistant<|end_header_id|>\\n\\n' }}"\
      "{% endif %}"
unsloth_eos_token = "eos_token"


tokenizer = get_chat_template(
    tokenizer,
    chat_template = (unsloth_template, unsloth_eos_token,), # You must provide a template and EOS token
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset(dataset_name, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

if debug:
  print("input:", dataset[5]["conversations"])
  print("output:", dataset[5]["text"])
  print("input:", dataset[4]["conversations"])
  print("output:", dataset[4]["text"])

Generating train split:   0%|          | 0/16 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

input: [{'role': 'user', 'content': 'Can you introduce yourself?'}, {'role': 'assistant', 'content': 'I am co-pilot, an AI assistant trained by Selector Software.'}]
output: <|begin_of_text|><|start_header_id|>user<|end_header_id|>

Can you introduce yourself?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I am co-pilot, an AI assistant trained by Selector Software.<|eot_id|>
input: [{'role': 'user', 'content': 'Do you have a name?'}, {'role': 'assistant', 'content': 'As an AI assistant developed by Selector Software, I got the name co-pilot.'}]
output: <|begin_of_text|><|start_header_id|>user<|end_header_id|>

Do you have a name?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

As an AI assistant developed by Selector Software, I got the name co-pilot.<|eot_id|>


# Start training

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 20,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)


trainer_stats = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:307: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was c

Map (num_proc=2):   0%|          | 0/16 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 16 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 20
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,4.701800
2,4.784700
3,4.663500
4,4.499500
5,3.940700
6,2.832100
7,2.141200
8,1.542300
9,1.294000
10,1.086200


# Save trained adapter

In [7]:
# Save locally
if save_local:
  model.save_pretrained("lora_local_model")
  tokenizer.save_pretrained("lora_local_model")

# Upload on hugging face
if len(upload_to_hf) != 0:
  model.push_to_hub(upload_to_hf, token = hf_token)
  tokenizer.push_to_hub(upload_to_hf, token = hf_token)

# Merge and Upload
# Merge to 16bit
if len(merge16_and_upload)!= 0:
  #model.save_pretrained_merged("lora_instruct_model", tokenizer, save_method = "merged_16bit",)
  model.push_to_hub_merged(merge16_and_upload, tokenizer, save_method = "merged_16bit", token = hf_token)

# Merge to 4bit
if len(merge4_and_upload)!= 0:
  #model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
  model.push_to_hub_merged(merge4_and_upload, tokenizer, save_method = "merged_4bit", token = hf_token)

# Just LoRA adapters
#if False:
  #model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
  #model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

README.md:   0%|          | 0.00/592 [00:00<?, ?B/s]

Saved model to https://huggingface.co/andy006/lora_instruct_model_id


Unsloth: You are pushing to hub, but you passed your HF username = andy006.
We shall truncate andy006/s2-oracle-trained to s2-oracle-trained
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 16.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 56.08 out of 83.48 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 57.63it/s]


...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/andy006/s2-oracle-trained


# Inference

In [9]:
from unsloth.chat_templates import get_chat_template

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Who are you?"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nWho are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nI am co-pilot, an AI assistant developed by Selector Software. How can I assist you today?<|eot_id|>']